## Loading fast5 data
From the ONT tutorial - https://epi2me.nanoporetech.com/notebooks/Introduction_to_Fast5_files.html

In [12]:
%load_ext autoreload
%autoreload 2

In [6]:
from epi2melabs import ping
tutorial_name = "fast5_tutorial"
pinger = ping.Pingu()
pinger.send_notebook_ping('start', tutorial_name)

# create a work directory and move into it
working_dir = 'epi2melabs/{}/'.format(tutorial_name)
%mkdir "$working_dir"
%cd "$working_dir"

c:\Users\Parv\Doc\RA\Projects\DSP\epi2melabs\fast5_tutorial


c:\Users\Parv\anaconda3\envs\pytorch_gpu\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [8]:
bucket = "ont-exd-int-s3-euwst1-epi2me-labs"
domain = "s3-eu-west-1.amazonaws.com"
site = "https://{}.{}".format(bucket, domain)
site = "https://ont-exd-int-s3-euwst1-epi2me-labs.s3-eu-west-1.amazonaws.com"

%rm -rf sample_fast5
%wget -O sample_fast5.tar $site/fast5_tutorial/sample_fast5.tar
%tar -xvf sample_fast5.tar
%wget -O fast5_sample.bam $site/fast5_tutorial/fast5_sample.bam
%wget -O fast5_sample.bam.bai $site/fast5_tutorial/fast5_sample.bam.bai

UsageError: Line magic function `%rm` not found.


In [10]:
import os
import requests
import tarfile
import shutil
from tqdm import tqdm

bucket = "ont-exd-int-s3-euwst1-epi2me-labs"
domain = "s3-eu-west-1.amazonaws.com"
site = f"https://{bucket}.{domain}"

# Ensure directory is clean
if os.path.exists("sample_fast5"):
    shutil.rmtree("sample_fast5")
os.makedirs("sample_fast5", exist_ok=True)

def download_file(url, filename):
    response = requests.get(url, stream=True)
    response.raise_for_status()
    total_size = int(response.headers.get("content-length", 0))
    block_size = 8192  # 8 KB
    desc = filename if len(filename) < 30 else filename[:27] + "..."
    progress = tqdm(total=total_size, unit="B", unit_scale=True, desc=desc)
    
    with open(filename, "wb") as f:
        for chunk in response.iter_content(block_size):
            f.write(chunk)
            progress.update(len(chunk))
    progress.close()

# Download the tar file
download_file(f"{site}/fast5_tutorial/sample_fast5.tar", "sample_fast5.tar")

# Extract tar file
with tarfile.open("sample_fast5.tar", "r") as tar:
    tar.extractall(path="sample_fast5")

# Download BAM and index files
download_file(f"{site}/fast5_tutorial/fast5_sample.bam", "fast5_sample.bam")
download_file(f"{site}/fast5_tutorial/fast5_sample.bam.bai", "fast5_sample.bam.bai")

print("All files downloaded and extracted.")


sample_fast5.tar: 100%|██████████| 2.52G/2.52G [05:03<00:00, 8.32MB/s]
fast5_sample.bam: 100%|██████████| 71.6M/71.6M [00:09<00:00, 7.45MB/s]
fast5_sample.bam.bai: 100%|██████████| 123k/123k [00:00<00:00, 3.18MB/s]

All files downloaded and extracted.


### Collecting sample data
Using the read ids to find corresponding sequencing data for fast5 - fastq
And then making a pickled dataset with all the best estimates

In [ ]:
from utils import get_data_from_fast5, get_fastq_records
import os
import gzip
import shutil
from tqdm import tqdm

In [35]:
def extract_tar_file(input_filepath):
    output_filepath = input_filepath[:-3]
    with gzip.open(input_filepath, "rb") as f_in:
        with open(output_filepath, "wb") as f_out:
            shutil.copyfileobj(f_in, f_out)
    

In [ ]:
# fastq data

fq_records_total = []

preceeding_path = r'epi2melabs\fast5_tutorial\sample_fast5\sample_fast5'
preceeding_path = os.path.join(os.getcwd(), preceeding_path)

for file in tqdm(os.listdir(preceeding_path)):
    if file.endswith('.gz'):
        full_filepath = os.path.join(preceeding_path, file)
        extract_tar_file(full_filepath)
        records = get_fastq_records(full_filepath[:-3])
        fq_records_total.append(records)



100%|██████████| 36/36 [00:08<00:00,  4.40it/s]


In [47]:
# fast5 data
squiggles_arr = []
read_ids_arr = []

preceeding_path = r'epi2melabs\fast5_tutorial\sample_fast5\sample_fast5\workspace'
preceeding_path = os.path.join(os.getcwd(), preceeding_path)

for file in tqdm(os.listdir(preceeding_path)):
    if file.endswith('.fast5'):
        full_filepath = os.path.join(preceeding_path, file)
        read_ids, squiggles = get_data_from_fast5(full_filepath)
        squiggles_arr.extend(squiggles)
        read_ids_arr.extend(read_ids)

100%|██████████| 5/5 [00:43<00:00,  8.65s/it]


In [48]:
len(squiggles_arr)

20000

In [49]:
fq_records[0]

SeqRecord(seq=Seq('CAGTGTACTTCGTTCAGTTACGTACTACTTGCCTGTCGCTCTACTTCGGCGTCT...CTT'), id='6ea6609b-2962-4ac4-a792-5375fb9b2d23', name='6ea6609b-2962-4ac4-a792-5375fb9b2d23', description='6ea6609b-2962-4ac4-a792-5375fb9b2d23 runid=2bf4f211a2e2d04662e50f27448cfd99dafbd7ee sampleid=DS_R941_M12_ZYMO read=40 ch=450 start_time=2019-01-15T15:54:54Z', dbxrefs=[])

In [50]:
# Gotta make a dataframe and then I can merge via read id

import pandas as pd

read_ids_fq = []
strands_fq = []

for record in fq_records:
    read_ids_fq.append(record.id)
    strands_fq.append(str(record.seq))


In [52]:
df_fq = pd.DataFrame({"read_id": read_ids_fq, "strand": strands_fq})  # fastq dataframe

In [54]:
df_f5 = pd.DataFrame({"read_id": read_ids_arr, "squiggle": squiggles_arr})

In [56]:
df = df_fq.merge(df_f5, on='read_id')

In [58]:
df.to_pickle("clean_f5_fq_data_ont.pkl")